HaRe
====

HaRe (Harassment Recognizer) is a command line tool and Python library to automatically detect harassment as it happens (real-time) with the help of machine learning techniques. In this notebook we will look at the Python library.

Basic usage: monitoring conversations
--------------------------------------------------------

The easiest way to use HaRe is by simply loading a pretrained HaRe model included with this repo in the models folder, like the one named 'simple':

In [1]:
from hare import load_pretrained

simple_hare = load_pretrained('hare/pretrained/simple')

You can then use this object to monitor conversations in progress. Let's start a conversation and ask HaRe to monitor it:

In [2]:
from hare import Conversation

convo = Conversation()
simple_hare.add_conversation(convo)

At any point in time, you can then request the current status of the conversation according to this HaRe model:

In [3]:
convo.add_utterance(speaker='a',content='hello')
convo.add_utterance(speaker='b',content='hi everyone')
simple_hare.get_status()

{'a': 0.8264739513397217, 'b': 0.8264739513397217}

You can also add multiple sentences at once (for example a whole conversation if it has already finished), and see what HaRe's best guess was at every moment in time.

In [10]:
from hare import Utterance

convo.add_utterances([Utterance(speaker='a',content='good luck'),
                          Utterance(speaker='c',content='ur all n00bs')])
simple_hare.visualize_history_for_conversation()

a: hello
{'a': 0.8264739513397217}
---

b: hi everyone
{'a': 0.8264739513397217, 'b': 0.8264739513397217}
---

a: good luck
{'a': 0.8264739513397217, 'b': 0.8264739513397217}
---

c: ur all n00bs
{'a': 0.8264739513397217, 'b': 0.8264739513397217, 'c': 0.8264739513397217}
---



If you add multiple conversations for Hare to monitor, you will need to specify the conversation index when asking for the status:

In [6]:
second_convo = Conversation()

second_convo.add_utterance(speaker='a',content='hello')
second_convo.add_utterance(speaker='b',content='hi everyone')

simple_hare.get_status(id=1)

{'a': 0.8264739513397217, 'b': 0.8264739513397217}

Evaluating 
----------
If you have a labeled dataset (that is: for each conversation an indication which participants are considered toxic), HaRe can calculate to what extent its judgments match the labels. A label can range from the default 0 (not toxic at all) to 1 (maximally toxic). Let's label speaker c:

In [8]:
convo.label_speaker('c',0.9)

There are several evaluation metrics, depending on what is important to you (detecting ALL harassment, detecting harassment quickly, no false positives, etc):

In [12]:
simple_hare.calculate_retrospective_accuracy()

0.2